# 0 : Import the required libraries and define the Salesforce connection 

In [14]:
from simple_salesforce import Salesforce
import pandas as pd
import requests
config = pd.read_json('config.json')
source_org = Salesforce(username=config['source']['username'], password=config['source']['password'], security_token=config['source']['token'])
connected = source_org.session_id is not None
print('Connected to Source: '+str(connected))

Connected to Source: True


# 1 : Query the Required data from Salesforce

In [15]:
query = "SELECT Id, Title, VersionData,FileExtension FROM ContentVersion"
content_dict = source_org.query_all(query)
if content_dict['totalSize'] == 0: 
    print('No ContentVersion records found')
else:
    print('ContentVersion records found')

ContentVersion records found


# 2 : Use the content VersionData to create a file URL and use Request to do a Http call to this url to download the file

In [16]:
for content in content_dict['records']:
    baseurl = "https://brave-impala-f8r87b-dev-ed.trailblaze.my.salesforce.com/"
    content_id = content['Id']
    title = content['Title']
    fileExtension = content['FileExtension']
    version_data_url = baseurl+content['VersionData']
    response = requests.get(version_data_url, headers={'Authorization': 'Bearer ' + source_org.session_id})
    if response.status_code == 200:
        with open( title+'.'+fileExtension, 'wb') as f:
            f.write(response.content)
        print('ContentVersion ' + title + ' downloaded successfully')
    else:
        print('ContentVersion ' + title + ' download failed')

ContentVersion thflogo downloaded successfully
